# Local Evaluation - Groundedness

After you have setup and configured the prompt flow, its time to evaluation its performance. Here we can use the prompt flow SDK to test different questions and see how the prompt flow performs using the evaluation prompt flows provided.

In [1]:
from promptflow import PFClient
pf_client = PFClient()

from dotenv import load_dotenv

from pathlib import Path
load_dotenv(Path("../local.env"))

True

In [2]:
# Add a question to test the base prompt flow.
question = "Can you tell me about your jackets?"
customerId = "4"
output = pf_client.test(
    flow="../contoso-chat", # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output["answer"] = "".join(list(output["answer"]))

[2024-01-11 21:22:58,290][promptflow._sdk.entities._connection][WARNING] - Please use connection.secrets[key] to access secrets.


2024-01-11 21:22:58 +0000   29173 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-11 21:22:58 +0000   29173 execution.flow     INFO     Start to run 5 nodes with concurrency level 16.
2024-01-11 21:22:58 +0000   29173 execution.flow     INFO     Executing node question_embedding. node run id: 63be177c-0801-4ba5-bbbf-90cf6f184011_question_embedding_0
2024-01-11 21:22:58 +0000   29173 execution.flow     INFO     Executing node customer_lookup. node run id: 63be177c-0801-4ba5-bbbf-90cf6f184011_customer_lookup_0
2024-01-11 21:22:59 +0000   29173 execution.flow     INFO     Node question_embedding completes.
2024-01-11 21:22:59 +0000   29173 execution.flow     INFO     Executing node retrieve_documentation. node run id: 63be177c-0801-4ba5-bbbf-90cf6f184011_retrieve_documentation_0
2024-01-11 21:22:59 +0000   29173 execution.flow     INFO     Node customer_lookup completes.
2024-01-11 21:23:00 +0000   29173 execution.flow     INFO     Node retrieve_documentatio

In [3]:
output

{'answer': 'Of course, Sarah Lee! Let me tell you about our jackets. 😄\n\nWe have two amazing jackets available at Contoso Outdoors:\n\n1. Summit Breeze Jacket: 🏔️\n   - Lightweight and perfect for outdoor adventures.\n   - Trail-ready, windproof design with water-resistant fabric.\n   - Breathable and comfortable for any activity.\n   - Features adjustable cuffs, zippered pockets, and reflective accents.\n   - ⭐ Recommended with the CozyNights Sleeping Bag for a cozy camping combo! ⛺\n\n2. RainGuard Hiking Jacket: ☔\n   - Ultimate solution for weatherproof comfort.\n   - Waterproof and breathable fabric for a dry and comfortable experience.\n   - Rugged construction and adjustable hood for durability and customization.\n   - Equipped with ventilation zippers for increased airflow.\n   - ⭐ Pair it with the TrailBlaze Hiking Pants for a stylish and functional hiking outfit! 🌲\n\nBoth jackets are designed for outdoor enthusiasts like you, Sarah Lee! Stay stylish, protected, and ready for

Test the groundedness of the prompt flow with the answer from the above question.

In [4]:
test = pf_client.test(
    flow="groundedness",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

2024-01-11 21:23:03 +0000   29173 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-11 21:23:03 +0000   29173 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-01-11 21:23:03 +0000   29173 execution.flow     INFO     Executing node groundedness_score. node run id: c00abf47-ac26-44da-974a-9d75ccd8592c_groundedness_score_0
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Node groundedness_score completes.
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Executing node concat_scores. node run id: c00abf47-ac26-44da-974a-9d75ccd8592c_concat_scores_0
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Node concat_scores completes.
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Start to run 1 nodes with concurrency level 16.
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Executing node aggregate_variants_results. node run id: 7bd1819f-555b-4125-ab0f-516087b8c603_aggre

In [5]:
test

{'gpt_groundedness': 5.0}

# Local Evaluation - Multiple Metrics 

Now use the same prompt flow and test it against the Multi Evaluation flow for groundedness, coherence, fluency, and relevance.

In [6]:
test_multi = pf_client.test(
    "multi_flow",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Start to run 8 nodes with concurrency level 16.
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Executing node coherence_score. node run id: c3fcb157-e400-4b54-9dda-6e7bf83c5b9b_coherence_score_0
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Executing node fluency_score. node run id: c3fcb157-e400-4b54-9dda-6e7bf83c5b9b_fluency_score_0
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Executing node groundedness_score. node run id: c3fcb157-e400-4b54-9dda-6e7bf83c5b9b_groundedness_score_0
2024-01-11 21:23:04 +0000   29173 execution.flow     INFO     Executing node relevance_score. node run id: c3fcb157-e400-4b54-9dda-6e7bf83c5b9b_relevance_score_0
2024-01-11 21:23:05 +0000   29173 execution.flow     INFO     Node coherence_score completes.
2024-01-11 21:23:05 +0000   29173 executi

In [7]:
test_multi

{'gpt_coherence': 5.0,
 'gpt_fluency': 5.0,
 'gpt_groundedness': 5.0,
 'gpt_relevance': 5.0}

# AI Studio Azure batch run on an evaluation json dataset

Now in order to test these more thoroughly, we can use the Azure AI Studio to run batches of test data with the evaluation prompt flow on a larger dataset.

In [8]:
import json
# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

In [9]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Populate the `config.json` file with the subscription_id, resource_group, and workspace_name.

In [10]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ../config.json


Add the runtime from the AI Studio that will be used for the cloud batch runs.

In [11]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
# load flow
flow = "../contoso-chat"
# load data
data = "../data/salestestdata.jsonl"

In [12]:
# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name = timestamp+"chat_base_run"
print(run_name)

2024_01_11_212319chat_base_run


Create a base run to use as the variant for the evaluation runs. 

_NOTE: If you get "'An existing connection was forcibly closed by the remote host'" run the cell again._

In [13]:
# create base run in Azure Ai Studio
base_run = pf_azure_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(base_run)

[2024-01-11 21:23:20,053][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and request may fail with timeout error. Wait a while and resubmit same flow can successfully start the run.
Uploading salestestdata.jsonl (< 1 MB): 100%|██████████| 465/465 [00:00<00:00, 2.77kB/s]


Uploading contoso-chat (0.66 MBs): 100%|██████████| 658668/658668 [00:01<00:00, 539675.80it/s] 


[2024-01-11 21:23:23,567][promptflow.azure._restclient.flow_service_caller][INFO] - Start polling until session creation is completed...


Waiting for session creation, current status: InProgress
Waiting for session creation, current status: InProgress
Waiting for session creation, current status: InProgress
Waiting for session creation, current status: InProgress
Waiting for session creation, current status: InProgress


[2024-01-11 21:26:53,907][promptflow.azure._restclient.flow_service_caller][INFO] - Session creation finished with status Succeeded.


Waiting for session creation, current status: Succeeded
Portal url: https://ai.azure.com/projectflows/bulkrun/run/2024_01_11_212319chat_base_run/details?wsid=/subscriptions/9cc85ebc-ba00-4394-9f22-9dbb99e76655/resourcegroups/rg-contosochatai/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-aiproj
name: 2024_01_11_212319chat_base_run
created_on: '2024-01-11T21:26:58.663258+00:00'
status: Preparing
display_name: 2024_01_11_212319chat_base_run
description: null
tags: {}
properties:
  azureml.promptflow.runtime_name: automatic
  azureml.promptflow.runtime_version: 20231218.v2
  azureml.promptflow.definition_file_name: flow.dag.yaml
  azureml.promptflow.session_id: b44b14768b8e054c96eb643d010076cd6087d3f708341631
  azureml.promptflow.flow_lineage_id: a6117409b8b717c7e79942d8573769a6d1f4ac180e7fabad302590d2b43bff3c
  azureml.promptflow.flow_definition_datastore_name: workspaceblobstore
  azureml.promptflow.flow_definition_blob_path: LocalUpload/39fadcf054410fe3c78303ee6edb

In [14]:
pf_azure_client.stream(base_run)

2024-01-11 21:27:04 +0000     165 promptflow-runtime INFO     [2024_01_11_212319chat_base_run] Receiving v2 bulk run request bc14fc76-d349-4cf6-b0af-ed9fc2785121: {"flow_id": "2024_01_11_212319chat_base_run", "flow_run_id": "2024_01_11_212319chat_base_run", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "9b2a6f52-96b8-454b-9c95-b07e156bb334"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://stcontosocha985657812778.blob.core.windows.net/9e083a5b-0e63-44ca-9d0e-fc0d4f0c2da4-azureml/ExperimentRun/dcid.2024_01_11_212319chat_base_run/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=64f4f677-3361-4d69-9fe7-39384cc6d2f9&sktid=d2f64b03-2cce-4f20-900e-b81248ebc1a8&skt=2024-01-11T21%3A16%3A56Z&ske=2024-01-13T05%3A26%3A56Z&sks=b&skv=2019-07-07&st=2024-01-11T21%3A17%3A03Z&se=2024-01-12T05%3A27%3A03Z&sp=rcw", "app_insights_instrumentation_key": "InstrumentationKey=**data_scrubbed**;IngestionEndpoin

In [15]:
details = pf_azure_client.get_details(base_run)
details.head(10)

,inputs.customerId,inputs.question,inputs.line_number,inputs.chat_history,outputs.answer,outputs.context
outputs.line_number,,,,,,
4,2,What gear do you recommend for hiking?,4,[],"Hey Jane Doe! For hiking gear, I recommend the...","[{'id': '10', 'title': 'TrailBlaze Hiking Pant..."
1,1,Do you have any climbing gear?,1,[],"Hey John! 🧗‍♂️ Absolutely, we have some awesom...","[{'id': '9', 'title': 'SummitClimber Backpack'..."
0,4,tell me about your hiking jackets,0,[],Hey Sarah Lee! 🌟 Let me tell you about our awe...,"[{'id': '17', 'title': 'RainGuard Hiking Jacke..."
2,3,Can you tell me about your selection of tents?,2,[],Hey Michael! 🌟 We've got a fantastic selection...,"[{'id': '15', 'title': 'SkyView 2-Person Tent'..."
3,6,Do you have any hiking boots?,3,[],"Hey Emily! 👋 Absolutely, we have some awesome ...","[{'id': '4', 'title': 'TrekReady Hiking Boots'..."


# Cloud Eval run on Json Data

In [16]:
eval_flow = "multi_flow/"
data = "../data/salestestdata.jsonl"
run_name = timestamp+"chat_eval_run"
print(run_name)

eval_run_variant = pf_azure_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=base_run,  # use run as the variant
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

2024_01_11_212319chat_eval_run


[2024-01-11 21:27:43,226][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and request may fail with timeout error. Wait a while and resubmit same flow can successfully start the run.
Uploading multi_flow (0.03 MBs): 100%|██████████| 31979/31979 [00:01<00:00, 21804.60it/s]




In [ ]:
pf_azure_client.stream(eval_run_variant)

In [ ]:
details = pf_azure_client.get_details(eval_run_variant)
details.head(10)

In [ ]:

metrics = pf_azure_client.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

In [ ]:
pf_azure_client.visualize([base_run, eval_run_variant])